#Sparse Subspace Clustering (Vidal IEEE Trans. on PAMI 2013)

**References:**


1.   Main Paper (Vidal IEEE Trans. on PAMI 2013): https://arxiv.org/pdf/1203.1005.pdf
2.   Supplementary Paper (Vidal CVPR 2009): http://cis.jhu.edu/~ehsan/Downloads/SSC-CVPR09-Ehsan.pdf
3.   Spectral Clustering: http://people.csail.mit.edu/dsontag/courses/ml14/notes/Luxburg07_tutorial_spectral_clustering.pdf
4.   Spectral Clustering Code: https://juanitorduz.github.io/spectral_clustering/











Our objective is given a set of points drawn from a `union of subspaces`, we need to find the number of subspaces, their dimensions, a basis for each subspace, and the segmentation of the data.

Let all the points be from $D$ dimensional space, here, for our use, we consider the value of $D$ to be $10$. Let us consider that we have $N=100$ $D$ dimensional data points.

In [127]:
import numpy as np
import cvxpy as cp
from cvxpy.atoms.elementwise.power import power

##### Set Parameters of the data space

In [128]:
N = 150 # Total no of points
D = 4  # Dimension of space
K = 3 # Number of clusters

In [129]:
from sklearn.datasets import load_iris
iris_data = load_iris()
iris_X = np.asarray(iris_data.data)
input_data = iris_X.T

In the $Y_i$ numpy array, we have $N_i$, $D$ dimensional data points i.e. the array $Y$ contains the collection of data points. We need to find out the value of $n$ which is the number of subspaces from which they have been sampled, $d_i$ which is the dimension of each subspace, $A_i$ which will be the basis for each subspace.

The matrix $Y$ contains the data points arranged sequentially forming a matrix of shape $(D,N)$. The $Y$ matrix is of the form $[y[0],y[1], \ldots, y[N-1]], $where each $y_i$ denotes the vector of data point.

In [130]:
# def find_sparse_sol(Y,i,N):
#   if i is 0:
#     Ybari = Y[:,1:N]
#   if i is N-1:
#     Ybari = Y[:,0:N-1]
#   if i!=0 and i!=N-1:
#     Ybari = np.concatenate((Y[:,0:i],Y[:,i+1:N]),axis=1)
#   yi = Y[:,i].reshape(D,1)
#   # this ci will contain the solution of the l1 optimisation problem  min |ci|1   yi = Ybari*ci
#   ci = cp.Variable(shape=(N-1,1))
#   constraint = [Ybari * ci == yi]
#   obj = cp.Minimize(cp.norm(ci,1))
#   prob = cp.Problem(obj, constraint)
#   prob.solve()
#   return ci.value
def find_sparse_sol(Y,i,N):
  if i is 0:
    Ybari = Y[:,1:N]
  if i is N-1:
    Ybari = Y[:,0:N-1]
  if i!=0 and i!=N-1:
    Ybari = np.concatenate((Y[:,0:i],Y[:,i+1:N]),axis=1)
  yi = Y[:,i].reshape(D,1)
  # this ci will contain the solution of the l1 optimisation problem  min |ci|1   yi = Ybari*ci
  ci = cp.Variable(shape=(N-1,1))
  constraint = [cp.sum(ci)==1] #Ybari * ci == yi, 
  obj = cp.Minimize(power(cp.norm(yi-Ybari*ci,2),2) + 0.082*cp.norm(ci,1))
  prob = cp.Problem(obj, constraint)
  prob.solve()
  return ci.value

In [131]:
ci = find_sparse_sol(input_data,2,N)
count = 0
for i in range(N-1):
  if ci[i,:]!=0:
    count += 1
print(count)

149


The function `find_sparse_sol` finds the coefficients for the equation $y_i == Y_{\hat{i}} * c_i$. The method will return the $c_i$ coefficient vector for the removal of the $y_i$ vector from the data matrix. This $c_i$ vector will contain non zero coefficients for all the other points which are in the same subspace as $y_i$. So, among the $N-1$ column entries, $N_i - 1$ entries will be non zero. The remaining entries will be $0$.

In [132]:
C = np.concatenate((np.zeros((1,1)),find_sparse_sol(input_data,0,N)),axis=0)

for i in range(1,N):
  ci = find_sparse_sol(input_data,i,N)
  zero_element = np.zeros((1,1))
  cif = np.concatenate((ci[0:i,:],zero_element,ci[i:N,:]),axis=0)
  C = np.concatenate((C,cif),axis=1)
print(C.shape)

(150, 150)


We now include a zero vector of size $(1,1)$ at the $i$ th position of $c_i$ to form $\hat{c_i}$ which we represent as *cif* in the code.
Then we concatenate $[\hat{c_1}$, $\hat{c_2}$, ..., $\hat{c_N}]$ to form the matrix $C$.  

$C$ is the *Matrix of Coefficients* and is of the form $C = [\hat{c_1}, \hat{c_2}, ..., \hat{c_N}] \in \mathbb{R}^{NXN}$.

In [133]:
Cp = np.zeros((C.shape[0], C.shape[1]))
for i in range(N):
  for j in range(N):
    Cp[i,j] = abs(C[i,j]) + abs(C[j,i])

print(Cp.shape)

(150, 150)


In [134]:
cz = 0
for i in range(Cp.shape[0]):
    for j in range(Cp.shape[1]):
        if Cp[i,j] < 1e-5:
            cz += 1
print(cz)

1644


In the above code block, we make $C$ a symmetric matrix by the operation $C_{ij} = |C_{ij}| + |C_{ji}|$. It is still a valid representation of the similarity since if $y_i$ can write itself as a linear combination of all the points in the same subspace including $y_j$, then $y_j$ can also be represented as a linear combination of all the other points belonging to the same subspace including $c_i$.

In [135]:
D = np.zeros((N,N))
sum_list=[]
for i in range(N):
  csum = np.sum(Cp[:,i],axis=0)
  sum_list.append(csum)

D = np.diag(sum_list)
print(D)

[[1.24708066 0.         0.         ... 0.         0.         0.        ]
 [0.         1.38384376 0.         ... 0.         0.         0.        ]
 [0.         0.         1.26239179 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.21430676 0.         0.        ]
 [0.         0.         0.         ... 0.         1.98337482 0.        ]
 [0.         0.         0.         ... 0.         0.         1.27050197]]


$D$ is the `degree matrix`. In this case, $D \in \mathbb{R}^{NxN}$ is a diagonal matrix with $D_{ii} = \sum_{j}C_{ij}$. 

In [136]:
L = np.subtract(D, Cp)
print(L.shape)

(150, 150)


This $L$ is the Laplacian matrix, which can be defined as $L = D - C$. Next, we calculate the `eigenvalues` and `eigenvectors` of the Laplacian matrix, which we will use for Spectral clustering of the data points. $L$ is a *positive, semi-definite matrix* this means all the eigenvalues of the matrix will be greater than equal to $0$.

### Perform Spectral Clustering with Laplacian Matrix L

In [137]:
from scipy import linalg

eigenvals, eigenvcts = linalg.eig(L)

eigenvals = np.real(eigenvals)
eigenvcts = np.real(eigenvcts)

eig = eigenvals.reshape((N,1))

In [138]:
eig_list=[]
for x in eig:
  eig_list.append(x[0])

eig_list.sort()
print(eig_list)

[1.3316666854801256e-15, 0.3811581158235885, 0.6604065241820902, 0.6754057727942127, 0.7225771343874836, 0.7607450498017398, 0.8218097216959224, 0.9044513332565574, 0.9192952629796827, 0.9701143232596151, 0.9945014175372842, 1.0169484278178418, 1.0384098835274393, 1.0582806865502363, 1.0859489990011792, 1.087225061248286, 1.1050558187523045, 1.1093262583934311, 1.113140150763878, 1.1350833237621403, 1.143517219423256, 1.1605993180801848, 1.1688699327700127, 1.1721358347826056, 1.1832239897428838, 1.1867991885515856, 1.1906640819879393, 1.192323208304266, 1.1941305112230047, 1.1994003226716334, 1.2004669216065813, 1.203959485707678, 1.2045276967425456, 1.205228768125977, 1.207238657303304, 1.2081349396542995, 1.2086536807049637, 1.2089358048154568, 1.2100016103707059, 1.2107633048021638, 1.2117879029784429, 1.2134001314246388, 1.2149554642350164, 1.215150266692887, 1.2159217522007706, 1.216736093841525, 1.2187231531465819, 1.2202962229781478, 1.221325590698924, 1.2222274712415135, 1.223

Here, we sort all the eigenvalues of the $L$ matrix. We find that the number of zero eigenvalues is $2$, which are the first $2$ eigenvalues, which means that the graph has $2$ `connected components`, which is the **same as the number of subspaces** from which the data have been collected. 

> **Thus, the multiplicity of the zero eigenvalues of the Laplacian Matrix $L$ is equal to the *number of connected components in the graph* which is same as the number of subspaces.**

Sort Eigen Values

In [139]:
eigenvals_sorted_indices = np.argsort(eigenvals)
eigenvals_sorted = eigenvals[eigenvals_sorted_indices]

In [140]:
indices = []
for i in range(0,K):
    ind = []
    print(eigenvals_sorted_indices[i])
    ind.append(eigenvals_sorted_indices[i])
    indices.append(np.asarray(ind))

24
25
36


In [141]:
indices = np.asarray(indices)

In [142]:
zero_eigenvals_index = np.array(indices)

Here, the indices are arranged according to their sorted order of values and the sorted eigen values are stored in *eigenvals_sorted*.

In [143]:
eigenvals[zero_eigenvals_index]

array([[1.33166669e-15],
       [3.81158116e-01],
       [6.60406524e-01]])

Here, we collect the indices of *the zero eigen values* (for convenience we consider values smaller than $10^{-5}$ to be equal to zero). \\
*The zero eigen values* are the displayed.

In [144]:
import pandas as pd

proj_df = pd.DataFrame(eigenvcts[:, zero_eigenvals_index.squeeze()])
proj_df.columns = ['v_' + str(c) for c in proj_df.columns]
proj_df.head()

,v_0,v_1,v_2
0,0.08165,-0.124575,0.034847
1,0.08165,-0.108455,0.082008
2,0.08165,-0.130572,0.004090
3,0.08165,-0.132622,0.032966
4,0.08165,-0.124779,-0.040015


Stack the Eigen Vectors corresponding to the zero Eigen Values in a dataframe *proj_df*. This can be thought of as a $N X Z$ matrix ($Z$ = number of zero eigen values) where the columns denote an eigen vector and the rows denote a data point.

Apply *K-Means Clustering* with $K = 3$.

In [145]:
from sklearn.cluster import KMeans

def run_k_means(df, n_clusters):
    k_means = KMeans(random_state=25, n_clusters=n_clusters)
    k_means.fit(df)
    cluster = k_means.predict(df)
    return cluster

cluster = run_k_means(proj_df, n_clusters=K)

*run_k_means* applies `K-Means Clustering` on *proj_df* with number of clusters = $2$. \\
The clustering of the data points is returned in the variable *'cluster'*.

Display clusters formed

In [146]:
print(cluster)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 0 2 2 2 0 0 2 2 0 2 0 2 2 0 2 0 2 2 2 0 2 2 2
 2 2 2 2 0 2 2 2 2 0 0 0 2 2 0 2 2 0 2 2 2 0 0 2 0 2 0 0 0 0 0 2 0 2 2 0 0
 0 0 0 0 0 0 0 2 2 0 0 2 2 0 2 0 0 0 2 2 0 0 2 2 2 0 0 0 0 0 0 0 0 0 0 2 0
 0 0]


As we can see, the data points have been clustered into two subspaces: $0$ and $1$ corresponding to the $2$ subspaces that we have considered.

In [147]:
c0 = 0
for l in cluster:
  if l == 0:
    c0 += 1
print(c0)

53


In [148]:
c1 = 0
for l in cluster:
  if l == 1:
    c1 += 1
print(c1)

50


In [149]:
c2 = 0
for l in cluster:
  if l == 2:
    c2 += 1
print(c2)

47


So we find that as expected $50$ data points have been labelled $0$, signifying that they belong to  the $1$st subspace, while the remaining $50$ points must belong to the $2$nd subspace. 

In [150]:
orig = iris_data.target

In [151]:
pred = np.asarray(cluster)

In [152]:
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import normalized_mutual_info_score
print("ARI = " + str(adjusted_rand_score(orig, pred)))
print("NMI = " + str(normalized_mutual_info_score(orig, pred)))

ARI = 0.5666502143559329
NMI = 0.6248401785062485
